In [ ]:
import requests as r
from bs4 import BeautifulSoup
import csv
import copy
import misc
import data_processing
from importlib import reload
import pandas as pd
import numpy as np
reload(data_processing)
reload(misc)
pd.set_option('display.max_columns', 100)




## Remove Rows with too Many Missing Data

In [ ]:
data = data_processing.read_data("raw_data_long.csv")

data = list(filter(lambda d: len(d.keys()) == len(list(filter(None, list(d.values())))) , data))
with open('training_data_long.csv', 'w') as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames=list(data[0].keys()))
    writer.writeheader()
    writer.writerows(data)
  

## Load and Clean Data Using Pandas (Removes Bad Characters in Column Titles, Removes Population Column)

In [ ]:
import pandas as pd
import re
with open('training_data_long.csv', 'r') as file:
    csv_reader = csv.DictReader(file)
    data = list(csv_reader)

df = pd.DataFrame(data)
df

In [ ]:


#columns = list(df.columns)
#columns = list(map(lambda x: re.sub(r"^\s", "", x), columns))
#columns = list(map(lambda x: re.sub(r"^\s", "", x), columns))
#columns = list(map(lambda x: re.sub(r"\s$", "", x), columns))
#columns = list(map(lambda x: re.sub(r"\s$", "", x), columns))
#columns = list(map(lambda x: re.sub(r"\s?.?/$", "", x), columns))
columns = df.columns
populations = [df['Population (persons) 1/YR-1'],
               df['Population (persons) 1/YR-2'],
               df['Population (persons) 1/YR-3']]
#df.drop(df[columns[-2]], axis = 1, inplace= True)
#df[columns[-2]] = population.iloc[:,0]
df.drop(columns = "Population (persons) 1/YR-1", inplace=True)
df.drop(columns = "Population (persons) 1/YR-2", inplace=True)
df.drop(columns = "Population (persons) 1/YR-3", inplace=True)
df.drop(columns = " Population (persons) 3/YR-1", inplace=True)
df.drop(columns = " Population (persons) 3/YR-2", inplace=True)
df.drop(columns = " Population (persons) 3/YR-3", inplace=True)

In [ ]:
df

## Adjust for inflation

In [ ]:
for i in df.index:
    try:
        year = int(df.iloc[i,1])
        df.iloc[i, 4:] = df.iloc[i, 4:].map(lambda x: int(x) * misc.cpi_dict[year])
    except Exception as e:
        print(e)
        pass
df

## Add Popultation Column Back

In [ ]:
df["Population YR-1"] = populations[0]
df["Population YR-2"] = populations[1]
df["Population YR-3"] = populations[2]

## Normalize Data

In [ ]:
for column in df.columns:
    try:
        if column != "Year":
            df[column] = df[column].map(lambda x: int(x))
            x_max = max(list(df[column]))
            x_min = min(list(df[column]))
            df[column] = df[column].map(lambda x: (x - x_min)/(x_max - x_min))
    except Exception as e:
        print(e)
df.to_csv("training_data_long.csv")

## PCA for Exploratory Data Analysis

In [ ]:
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D 

%matplotlib widget 

target, data = data_processing.read_data("training_data_long.csv", mode = "List")
print(data.shape)

pca = PCA()
pca.fit(data)
#print(pca.explained_variance_ratio_)
pca.components_
transformed = pca.transform(data)
xs=transformed[:,0]
ys=transformed[:,1]
zs =transformed[:,2]

fig = plt.figure(figsize=(10, 10)) 
Axes3D(fig) 
ax = fig.add_subplot(projection='3d')
fig.patch.set_facecolor('white')
target = np.array(target)
indices = np.where(target == 'D')
ax.scatter(xs[indices], ys[indices], zs[indices], c="Blue", s=0.05, alpha = 0.3)
indices = np.where(target == 'R')
ax.scatter(xs[indices], ys[indices], zs[indices], c="Red", s=0.05, alpha = 0.3)

plt.xlabel("First Principal Component",fontsize=14)
plt.ylabel("Second Principal Component",fontsize=14)
plt.legend()
plt.show()


## Load Data, Split Data into Training/Valid/Test Sets

In [ ]:
import random
dict_list = data_processing.read_data("training_data_long.csv")
data = []
for _ in dict_list:
    data += [list(_.values())[1:]]
random.shuffle(data)
valid_set = data[-1000:]
test_set = data[:1000]
training_set = data[1000:-1000]


## Neural Net Setup

In [ ]:
import training
reload(training)

In [ ]:
nn = training.get_nn()
nn = training.train(nn, training_set, 0.0002, 1000, 10000)


In [ ]:
nn

In [ ]:
torch.set_printoptions(edgeitems=100)
tensors = data_to_tensor(training_set)
data = tensors.data
labels = tensors.labels
torch.sum(torch.vmap(torch.argmax)(labels) == torch.vmap(torch.argmax)(nn(data)))/len(labels)
